In [14]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Reshape

%matplotlib inline
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_parquet('abc.parquet')

In [3]:
df.drop(df.columns[[0, 1, 3, 4]], axis=1, inplace=True)
dataset = df.values

In [12]:
# Variables
BATCH_SIZE = 20
EPOCHS = 1
DROPOUT = 0.25
LSTM_UNITS = 100
TRAIN_LIMIT = 1.0
DAYS = 10

In [5]:
# Data split
cut_off = int(np.floor(TRAIN_LIMIT * dataset.shape[0]))
training = dataset[:cut_off, :]
testing = dataset[cut_off - BATCH_SIZE:, :]

In [6]:
# Scale Training
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(training)
test_scaled = scaler.transform(testing)

In [7]:
# Batch up data
x_train, y_train = [], []
for i in range(BATCH_SIZE, len(training)):
    x_train.append(train_scaled[i - BATCH_SIZE:i, :-1])
    y_train.append(train_scaled[i, -1])
x_train, y_train = np.array(x_train), np.array(y_train)

In [20]:
# Define model
model = Sequential()
model.add(LSTM(units=LSTM_UNITS, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(units=LSTM_UNITS))
# model.add(Dropout(DROPOUT))
# model.add(Dense(512, activation='relu'))
model.add(Dense(DAYS * x_train.shape[2]))
model.add(Reshape((DAYS, x_train.shape[2])))

In [21]:
# Run training
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

ValueError: Error when checking target: expected reshape_3 to have 3 dimensions, but got array with shape (2356, 1)

In [ ]:
# Batch up test data
x_test = []
for i in range(BATCH_SIZE, test_scaled.shape[0]):
    x_test.append(test_scaled[i - BATCH_SIZE:i, :-1])
x_test = np.array(x_test)

In [ ]:
# Get prediction
y_test = model.predict(x_test)
x_test = x_test[:, 0, :]
prediction = np.concatenate((x_test, y_test), axis=1)
prediction = scaler.inverse_transform(prediction)

In [ ]:
# Calc RMS
rms = np.sqrt(np.mean(np.power((testing[BATCH_SIZE:, -1] - prediction[:, -1]), 2)))
rms

In [ ]:
# Plot the graph
training = df[:cut_off]
testing = df[cut_off:]
testing['prediction'] = prediction[:, -1]
plt.figure(figsize=(30, 15)) 
plt.plot(training['dwt'])
plt.plot(testing[['dwt', 'prediction']])
plt.show()